# Gene Expression Among acI Clades: Mapping Metatranscriptomes to Composite Genomes

## Overview

We believe that reverse ecology metrics are sensitive to genome completeness. We also observed that--for a single tribe--at least four SAGs or MAGs were required before the pan-genome was "complete", based on single-copy marker genes. Because we only have four genomes for two tribes, we cannot confidently perform reverse ecology analysis at the tribe-level. At the clade level, we have 12 genomes from clade acI-A, 21 from clade acI-B, and 3 from acI-C. This should allow us to make confident predictions about the differences between the acI-A and acI-B clades.

We decided to map metatranscriptome samples to the "pan-genome" of each clade. To construct the pan-genome, we used our reference genome collection to define actinobacterial COGs (clusters of orthologous groups), and defined the pan-genome of a clade as the union of all COGs present in at least one genome. We then used BBMap to uncompetitively map metatranscriptome reads, and counted the unique reads which map to each actinobacterial COG.

### Identification of Actinobacterial COGs

For computational details, please see my [OrthoMCL repo](https://github.com/joshamilton/OrthoMCL), which will be merged into this one at a later date.

We used OrthoMCL to identify clusters of orthologous genes (COGs) in a set of 72 freshwater Actinobacterial genomes. OrthoMCL is an algorithm for grouping proteins into orthologous gene families based on sequence similarity. OrthoMCL takes as input a set of protein sequences and returns a list of COGs and the proteins which belong to each COG. The OrthoMCL pipeline consists of the following steps:

1. Convert KBase-annotated genomes (in `Genbank` format) to fasta amino acid (`faa`) format.
2. Format `faa` files to be compatible with OrthoMCL (script `01faaParser.py`).
3. Run all-vs-all BLAST on the concatenated set of protein sequences  (script `02parallelBlast`).
4. Initialize the MySQL server to store OrthoMCL output and run OrthoMCL (scripts `setupMySql.sh` and `runOrthoMCL.sh`).
5. Rearrange the OrthoMCL output into a user-friendly format (script `05parseCOGs`). The output of this script is described below.

#### cogTable.csv
A table listing the locus tags associated with each (genome, COG) pair.

|   | AAA023D18 | AAA023J06 | AAA024D14 |
|---|---|---|---|---|
| group00000 | AAA023D18.genome.CDS.1002; AAA023D18.genome.CDS.925; AAA023D18.genome.CDS.939 | AAA023J06.genome.CDS.1227; AAA023J06.genome.CDS.862	 |  |
| group00001 | AAA023D18.genome.CDS.800 | AAA023J06.genome.CDS.798 | AAA024D14.genome.CDS.945; AAA024D14.genome.CDS.1601 |

For example, in genome AAA023D18, the following genes belong to cog00000: AAA023D18.genome.CDS.1002, AAA023D18.genome.CDS.925, and AAA023D18.genome.CDS.939.

#### annotTable.csv
A table listing the annotations associated with each (genome, COG) pair.

|   | AAA023D18 | AAA023J06 | AAA024D14 |
|---|---|---|---|---|
| group00000 | Short-chain dehydrogenase/reductase in hypothetical Actinobacterial gene cluster; hypothetical protein; 3-oxoacyl-[acyl-carrier protein] reductase (EC 1.1.1.100) | 3-oxoacyl-[acyl-carrier protein] reductase (EC 1.1.1.100); 3-oxoacyl-[acyl-carrier protein] reductase (EC 1.1.1.100)	|  |
| group00001 | DNA gyrase subunit A (EC 5.99.1.3) | DNA gyrase subunit A (EC 5.99.1.3) | DNA gyrase subunit A (EC 5.99.1.3); Topoisomerase IV subunit A (EC 5.99.1.-) |

#### annotSummary.csv
This table provides a list of all annotations associated with the genes in a COG. It can be further manually parsed to reveal the distribution of annotations associated with a COG. For example, COG00000 contains 94 genes across 72 genomes, as follows:

| Annotation | Counts |
|------------|--------|
| 3-oxoacyl-[acyl-carrier protein] reductase (EC 1.1.1.100)	| 63 |
| Short-chain dehydrogenase/reductase in hypothetical Actinobacterial gene cluster | 11 |
| None Provided	| 9 |
| hypothetical protein | 3 |
| COG1028: Dehydrogenases with different specificities (related to short-chain alcohol dehydrogenases) | 2 |
| 2,3-butanediol dehydrogenase, S-alcohol forming, (S)-acetoin-specific (EC 1.1.1.76) | 1 |
| D-beta-hydroxybutyrate dehydrogenase (EC 1.1.1.30) | 1 |
| Oxidoreductase, short chain dehydrogenase/reductase family | 1 |
| Short-chain dehydrogenase/reductase SDR | 1 |
| Acetoacetyl-CoA reductase (EC 1.1.1.36) | 1 |
| short chain dehydrogenase | 1 |

This COG appears to be a 3-oxoacyl-[acyl-carrier protein] reductase.

#### genomeCOGs.csv
This file contains a list of (gene, COG) pairs, giving the COG associated with each gene in the genome. One such file is created per genome. For example,

    AAA023D18.genome.CDS.834,group01620
    AAA023D18.genome.CDS.1427,group00803

### References
1. Li, L., Stoeckert, C. J., & Roos, D. S. (2003). OrthoMCL: identification of ortholog groups for eukaryotic genomes. Genome Research, 13(9), 2178–89. http://doi.org/10.1101/gr.1224503

## Mapping of Metatranscriptome Reads

### Acquisition and QC
We have obtained metatranscriptome samples from three different experiments which serve as the basis for our mapping.

* Lake Mendota - Three samples collected during OMD-TOIL. These are not true biological replicates as they were collected at different time points, but we are treating them as such. Additional documentation can be found in the [OMD-TOIL Github repo] (https://github.com/McMahonLab/OMD-TOILv2).

* Amazon River - Twelve samples taken from six stations along the Amazon River (two biological replicates each station). For ease of analysis, I am currently analyzing these as twelve biological replicates.

* Lake Lanier - Four biological replicates.

These samples need to be processed prior mapping. 

The Lake Mendota and Lake Lanier MTs were processed as described in the [OMD-TOIL repo](https://github.com/McMahonLab/OMD-TOILv2): 
* Reads were quality filtered and merged using FLASH
* rRNA reads were removed using SortMeRNA

The Amazon River MTs were obtained directly from the authors, having already been processed as described in their manuscript.

Once processed, the reads were placed in `metaTranscripts\rawData\sample.fastq` and mapped as follows.

1. Magoc, T., & Salzberg, S. L. (2011). FLASH: fast length adjustment of short reads to improve genome assemblies. Bioinformatics, 27(21), 2957–2963. http://doi.org/10.1093/bioinformatics/btr507
2. Kopylova, E., Noe, L., & Touzet, H. (2012). SortMeRNA: fast and accurate filtering of ribosomal RNAs in metatranscriptomic data. Bioinformatics, 28(24), 3211–3217. http://doi.org/10.1093/bioinformatics/bts611
3. Satinsky, B. M., Fortunato, C. S., Doherty, M., Smith, C. B., Sharma, S., Ward, N. D., … Crump, B. C. (2015). Metagenomic and metatranscriptomic inventories of the lower Amazon River, May 2011. Microbiome, 3, 39. http://doi.org/10.1186/s40168-015-0099-0
4. Tsementzi, D., Poretsky, R. S., Rodriguez-R, L. M., Luo, C., & Konstantinidis, K. T. (2014). Evaluation of metatranscriptomic protocols and application to the study of freshwater microbial communities. Environmental Microbiology Reports, 6(6), 640–655. http://doi.org/10.1111/1758-2229.12180

### Mapping of Reads: Uncompetitive

This section describes the steps to map metatranscriptomic reads to the reference genomes using BBMap. In this approach, each metatranscriptomic sample is mapped to a single `fasta` file containing all reference genomes. Mapping is performed such that a read is allowed to map equally well to multiple sites within a genome. In the next section, we aggregate all reads which map to a single actinoCOG (in any genome) and count the total number of unique reads. 

#### Creation of composite genome
Downstream counting of reads requires the `fasta` and `gff` files to be compatible. The `gff` files I have created use the KBase contig naming scheme: `>genome.contig.#`. I used KBase-formated fna files (`refGenomes\fna-kbase`) to create a single `fasta` file containg all acI genomes. This genome is located in `refGenomes\concat\acI.fna`.

#### Mapping
For convenience, the script `metaTranscripts\scripts\01uncompReadMapping.pl` will execute the pipeline with a single command. The script takes as input the directories described below, and maps each metatranscriptome to the concatenated acI with the following command:

    bbmap.sh t=numThreads in=sample.fastq outm=sample-genome.sam ref=genome.fna ambig=all nodisk sam=1.3

The script is called as follows:

    `perl 01uncompReadMapping.pl`

The script performs mapping in parallel, with 24 threads allocated to each mapping run. The script also specifies use of 1 thread per run, parallelized across 24 processors. For some reason, multithreading with BBMap causes `Zissou` to crash.

__Note:__ The script is currently hard-coded to use the folder structure described in this repo. 

The inputs to the wrapper function are as follows:

| Argument | Description  |
|---|---|
| mtFolder | location of the metatranscriptome reads to be mapped |
| genomeFolder | location of the reference genomes |
| mapFolder | desired output location of the mapped reads |
| numThreads | number of processors to use for mapping |
| numChildren | number of parallel mapping runs |

## Differential Expression Analysis

Finally, I plan to use edgeR or DESeq to identify COGs which show differential expression across the acI clades. I hypothesize that such genes will be associated with metabolic machinery (such as transporters) required to uptake and metabolize seed compounds unique to individual clades. DESeq and edgeR rely on biological replicates to test for statistically significant differences in expression. In this study, we used three sets of "replicates", as follows:

* Lake Mendota - Three samples collected during OMD-TOIL. These are not true biological replicates as they were collected at different time points, but we are treating them as such. Additional documentation can be found in the [OMD-TOIL Github repo] (https://github.com/McMahonLab/OMD-TOILv2).

* Amazon River - Twelve samples taken from six stations along the Amazon River (two biological replicates each station). For ease of analysis, I am currently analyzing these as twelve biological replicates.

* Lake Lanier - Four biological replicates.

I am currently analyzing these data and have nothing to put here.

### References
1. Anders, S., & Huber, W. (2010). Differential expression analysis for sequence count data. Genome Biology, 11, R106. http://doi.org/10.1186/gb-2010-11-10-r106
2. McCarthy, D. J., Chen, Y., & Smyth, G. K. (2012). Differential expression analysis of multifactor RNA-Seq experiments with respect to biological variation. Nucleic Acids Research, 40(10), 4288–4297. http://doi.org/10.1093/nar/gks042
3. Satinsky, B. M., Fortunato, C. S., Doherty, M., Smith, C. B., Sharma, S., Ward, N. D., … Crump, B. C. (2015). Metagenomic and metatranscriptomic inventories of the lower Amazon River, May 2011. Microbiome, 3, 39. http://doi.org/10.1186/s40168-015-0099-0
4. Tsementzi, D., Poretsky, R. S., Rodriguez-R, L. M., Luo, C., & Konstantinidis, K. T. (2014). Evaluation of metatranscriptomic protocols and application to the study of freshwater microbial communities. Environmental Microbiology Reports, 6(6), 640–655. http://doi.org/10.1111/1758-2229.12180

## Network Visualization

To generate some ideas for validating network predictions, I visualized the metabolic network of each clade. For simplicity, I plotted the condensation of the metabolic network graph. In the condensation, each node represents a single strongly connected component (SCC), a set of nodes such that every node is reachable from every other node.

In the schematic below, each circle represents an SCC. Each acI clade shares a similar structure, with many seed compounds (blue) pointing to a single giant component (green). In turn, the giant component has many outward-directed arcs to compounds which are not further metabolized (orange).

![Bowtie Network](files/imageFiles/bowtie.png)

Given this structure, I propose to validate reverse ecology predictions as described below.

## Validation

### Seed Compounds and the Giant SCC
A strongly connected component represents a set of metabolites which are "mutually synthesizable." From any metabolite in the SCC, all others can be synthesized. The seed compounds can be thought of as starting points for this "synthesis cascade," as they enable the cell to synthesize different compounds in the giant SCC. Thus, I propose to identify seed compounds which point to the giant SCC. If a clade has a unique such compound, I anticipate the genes to be expressed, as this compound represents a unique aspect of the clade's niche, and allows it to avoid competition. Likewise, if two clades share a compound, the two clades might compete for it. However, because all compounds which connect to the giant SCC are "equivalent," I anticipate that different clades will prefer different compounds.


### Complementary Metabolites
Here, I plan to first focus on the orange compounds in the above figure. These compounds are good candidates for metabolic exchange, as the donor has no further biosynthetic use for them. I plan to look at the activity of genes responsible for synthesizing these compounds in the donor, and consuming them in the recipient. If both genes are expressed, these compounds are candidates for further screening.

If no such metabolites arise, I propose to extend the analysis to all compounds which can be synthesized by the donor and consumed by the recipient.

### Splitting Compounds Into Groups

To perform the above analysis, I split all the compounds in a metabolic network into five sets:
* Seed compounds which point to the GCC
* Seed compounds which DON'T point to the GCC
* Sink compounds which are produced from the GCC
* Sink compounds whcih are produced elsewhere
* Remaining compounds

I will begin my analysis with the first and third groups, as the second and fourth contain only a few metabolites each.

The code below computes the above sets of compounds for each clade.

In [5]:
# Import special features for iPython
import sys
sys.path.append('../Python')

# Import Python modules 
# These custom-written modules should have been included with the package
# distribution. 
import graphFunctions as gf
import metadataFunctions as mf
import mtFunctions as mt

In [5]:
# Define local folder structure for data input and processing.
mergedModelDir = 'MergedData'
taxonFile = '../ExternalData/taxonomy.csv'

gf.defineMetabGroups(mergedModelDir, 'Lineage', taxonFile)
gf.defineMetabGroups(mergedModelDir, 'Clade', taxonFile)
gf.defineMetabGroups(mergedModelDir, 'Tribe', taxonFile)

### Mapping Compounds to COGs

The first step in this analysis maps compounds to COGs, based on the gene-to-reaction annotations in the metabolic networks of the individual genomes. The workflow for seed compounds is shown below, and a similar mapping occurs for sink compounds.

![Mapping of SEEDs to COGs](files/imageFiles/SeedsToCogs.png)

The Python code below performs this mapping.

__Note__: Mapping of COGs to compounds which are neither seeds nor sinks is more complicated, because we want to identify only COGs of syntheit reactions.

In [15]:
# Define local folder structure for data input and processing.
genomeModelDir = 'ProcessedModelFiles'
mergedModelDir = 'MergedData'
taxonFile = '../ExternalData/taxonomy.csv'

# Import the list of models
modelList = mf.getDirList('../'+mergedModelDir)

metabType = 'seedsToGCC'
mt.compoundsToCOGs(modelList, genomeModelDir, mergedModelDir, taxonFile, metabType)

metabType = 'sinkFromGCC'
mt.compoundsToCOGs(modelList, genomeModelDir, mergedModelDir, taxonFile, metabType)

### Mapping COGs to Expression Profiles

The next step is to extract expression scores for each of these COGs in each genome, based on expression values computed as described above. The Python code below performs this mapping.

In [17]:
# Define local folder structure for data input and processing.
rpkmDir = 'ExternalData/rpkmCounts'
mergedModelDir = 'MergedData'
taxonFile = '../ExternalData/taxonomy.csv'

# Import the list of models
modelList = mf.getDirList('../'+mergedModelDir)

metabType = 'seedsToGCC'
mt.COGsToProfiles(modelList, mergedModelDir, rpkmDir, metabType)

metabType = 'sinkFromGCC'
mt.COGsToProfiles(modelList, mergedModelDir, rpkmDir, metabType)